In [2]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import matplotlib
from skimage.transform import resize
from imageio import imread, imwrite
import pandas as pd
import glob
import pickle
import keras.backend as K

from stn.conv_model import conv_model
from stn.conv_model import conv_model_no_color_adjust
from sklearn.utils import resample
from lib.utils import load_gtsrb
from keras.metrics import sparse_categorical_accuracy

from cleverhans.utils import set_log_level
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.utils_tf import model_eval
import logging

from parameters import *
from small_net import *

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(sess)

In [4]:
signnames = pd.read_csv(DATA_DIR + 'signnames.csv')
X_train, y_train, X_val, y_val, X_test, y_test = load_gtsrb()

In [5]:
model = conv_model_no_color_adjust()
# model = conv_model()
model.load_weights("./keras_weights/stn_v5.hdf5")

get_stn_output = K.function([model.layers[0].input, K.learning_phase()],
                            [model.layers[1].output])
get_stn_output_v2 = K.function([model.layers[0].input], [model.layers[1].output])
stn_weight = model.layers[1].get_weights()

In [6]:
def get_patch(X, pos):
    """
    Crop all samples in <X> to make a set of patches <X_cropped>
    at the same position <pos>
    """
    
    top, bot, left, right = pos
    X_cropped = np.zeros((X.shape[0], bot - top, right - left, X.shape[3]))
    for i, x in enumerate(X):
        X_cropped[i] = np.copy(x[top:bot, left:right, :])
    return X_cropped

# Define cropping positions
pos_2 = (7, 23, 5, 17)
pos_0 = (7, 24, 15, 27) # Good for class 0 - 5
pos_3 = (7, 24, 6, 17)

pos_S = (7, 24, 0, 11)
pos_T = (7, 23, 8, 17)
pos_O = (6, 23, 13, 24)
pos_P = (6, 23, 22, 32)

pos_y11 = (11, 27, 9, 21)

In [7]:
X_train_bal, y_train_bal = gen_balance_data(X_train, y_train, [14], r=1)
y_train_bal = y_train_bal[:, np.newaxis]
X_val_bal, y_val_bal = gen_balance_data(X_val, y_val, [14], r=1)
y_val_bal = y_val_bal[:, np.newaxis]
X_test_bal, y_test_bal = gen_balance_data(X_test, y_test, [14], r=1)
y_test_bal = y_test_bal[:, np.newaxis]

y_test_stn = np.zeros([len(y_test), 1])
y_test_stn[np.where(y_test == 14)] = 1

In [13]:
n_attack = 250
ind_1 = np.where(y_test == 1)[0]
X_atk = X_test[ind_1][:n_attack]

y_target = np.zeros((n_attack, )) + 14
y_target = to_categorical(y_target, 43)

---

In [7]:
from feat_net import FeatNet

featnet = FeatNet("featnet_v7", [32, 32, 3], [1], 
                  [pos_S, pos_T, pos_O, pos_P], 
                  learning_rate=5e-4, 
                  save_path="model/featnet_v7.h5",
                  load_model=True,
                  stn_weight=stn_weight)

# data = (X_train_bal, y_train_bal, X_val_bal, y_val_bal)
# featnet.train_model(sess, data, n_epoch=10, batch_size=128)

In [8]:
featnet.eval_model(sess, (X_test_bal, y_test_bal))

(0.9911003236245954, -1.9728884115284702)

In [15]:
keras.backend.set_learning_phase(0)
set_log_level(logging.DEBUG)

# CarliniWagner attack
from lib.custom_cw import CustomCarliniWagnerL2

attack_iterations = 200
cw_params = {'binary_search_steps': 3,
             'max_iterations': attack_iterations,
             'learning_rate': 0.01,
             'batch_size': n_attack,
             'initial_const': 1,
             'y_target': y_target}
wrap_clf = KerasModelWrapper(model)
cw = CustomCarliniWagnerL2(wrap_clf, featnet, thres=4*.75, sess=sess)
adv = cw.generate_np(X_atk, **cw_params)

[INFO 2018-10-30 05:35:42,293 cleverhans] Constructing new graph for attack CustomCarliniWagnerL2
[DEBUG 2018-10-30 05:35:43,900 cleverhans] Running CWL2 attack on instance 0 of 250
[DEBUG 2018-10-30 05:35:44,038 cleverhans]   Binary search step 0 of 3
[DEBUG 2018-10-30 05:35:46,414 cleverhans]     Iteration 0 of 200: loss=342 l2=0 loss1=5.89e-10
[DEBUG 2018-10-30 05:35:47,791 cleverhans]     Iteration 20 of 200: loss=90.7 l2=5.26 loss1=2.32
[DEBUG 2018-10-30 05:35:49,100 cleverhans]     Iteration 40 of 200: loss=42.3 l2=9.4 loss1=3.36
[DEBUG 2018-10-30 05:35:50,361 cleverhans]     Iteration 60 of 200: loss=25.2 l2=10.1 loss1=3.65
[DEBUG 2018-10-30 05:35:51,676 cleverhans]     Iteration 80 of 200: loss=17.4 l2=9.54 loss1=3.8
[DEBUG 2018-10-30 05:35:53,061 cleverhans]     Iteration 100 of 200: loss=13.8 l2=8.57 loss1=3.85
[DEBUG 2018-10-30 05:35:54,468 cleverhans]     Iteration 120 of 200: loss=11.3 l2=7.68 loss1=3.84
[DEBUG 2018-10-30 05:35:55,892 cleverhans]     Iteration 140 of 200: 

In [27]:
from feat_net import FeatNet

featnet = FeatNet("featnet_v7", [32, 32, 3], [1], 
                  [pos_S, pos_T, pos_O, pos_P], 
                  learning_rate=5e-4, 
                  squeeze=5,
                  save_path="model/featnet_v7.h5",
                  load_model=True,
                  stn_weight=stn_weight)

In [28]:
featnet.eval_model(sess, (X_test_bal, y_test_bal))

(0.989967637540453, -1.9631902137524102)

In [29]:
y_adv = featnet.predict_model(sess, adv)
print(np.sum(y_adv >= 3))

240


In [21]:
from feat_net import FeatNet

featnet = FeatNet("featnet_v7", [32, 32, 3], [1], 
                  [pos_S, pos_T, pos_O, pos_P], 
                  learning_rate=5e-4, 
                  squeeze=4,
                  save_path="model/featnet_v7.h5",
                  load_model=True,
                  stn_weight=stn_weight)

In [22]:
featnet.eval_model(sess, (X_test_bal, y_test_bal))

(0.9824433656957928, -1.9451070731415332)

In [23]:
y_adv = featnet.predict_model(sess, adv)
print(np.sum(y_adv >= 3))

231


In [24]:
from feat_net import FeatNet

featnet = FeatNet("featnet_v7", [32, 32, 3], [1], 
                  [pos_S, pos_T, pos_O, pos_P], 
                  learning_rate=5e-4, 
                  squeeze=3,
                  save_path="model/featnet_v7.h5",
                  load_model=True,
                  stn_weight=stn_weight)

In [25]:
featnet.eval_model(sess, (X_test_bal, y_test_bal))

(0.9114886731391586, -1.758974016407161)

In [26]:
y_adv = featnet.predict_model(sess, adv)
print(np.sum(y_adv >= 3))

197


- Load squeeze trained net into non-squeeze
- Attack
- Put adv back to squeeze net

In [17]:
from feat_net import FeatNet

# Train squeeze net

featnet = FeatNet("featnet_sq3", [32, 32, 3], [1], 
                  [pos_S, pos_T, pos_O, pos_P], 
                  squeeze=3,
                  learning_rate=1e-4, 
                  save_path="model/featnet_sq3.h5",
                  load_model=False,
                  stn_weight=stn_weight)
data = (X_train_bal, y_train_bal, X_val_bal, y_val_bal)
featnet.train_model(sess, data, n_epoch=10, batch_size=128)

============= EPOCH: 0 =============
STEP: 0 	Loss: -0.2536
STEP: 50 	Loss: -1.1737
STEP: 100 	Loss: -2.1777
STEP: 150 	Loss: -2.1266
STEP: 200 	Loss: -1.9233
STEP: 250 	Loss: -1.7993
STEP: 300 	Loss: -1.8706
STEP: 350 	Loss: -1.9724
STEP: 400 	Loss: -1.6781
STEP: 450 	Loss: -2.1602
STEP: 500 	Loss: -1.8337
Train Acc|Loss:	0.9989|-1.9636
Val Acc|Loss:	0.9387|-1.8176
============= EPOCH: 1 =============
STEP: 0 	Loss: -1.8225
STEP: 50 	Loss: -2.1838
STEP: 100 	Loss: -2.3752
STEP: 150 	Loss: -2.0441
STEP: 200 	Loss: -2.1847
STEP: 250 	Loss: -1.5982
STEP: 300 	Loss: -1.8074
STEP: 350 	Loss: -2.4378
STEP: 400 	Loss: -2.0090
STEP: 450 	Loss: -2.0743
STEP: 500 	Loss: -1.9297
Train Acc|Loss:	0.9990|-1.9818
Val Acc|Loss:	0.9388|-1.8218
============= EPOCH: 2 =============
STEP: 0 	Loss: -2.0174
STEP: 50 	Loss: -2.1091
STEP: 100 	Loss: -2.0869
STEP: 150 	Loss: -2.0139
STEP: 200 	Loss: -1.9030
STEP: 250 	Loss: -1.8266
STEP: 300 	Loss: -2.1546
STEP: 350 	Loss: -2.1644
STEP: 400 	Loss: -2.0620
STE

In [18]:
featnet.eval_model(sess, (X_test_bal, y_test_bal))

(0.9515372168284789, -1.8816935792710017)

In [19]:
# Load squeeze trained net into non-squeeze

featnet = FeatNet("featnet_sq3", [32, 32, 3], [1], 
                  [pos_S, pos_T, pos_O, pos_P], 
                  squeeze=None,
                  learning_rate=5e-4, 
                  save_path="model/featnet_sq3.h5",
                  load_model=True,
                  stn_weight=stn_weight)

In [20]:
featnet.eval_model(sess, (X_test_bal, y_test_bal))

(0.9234223300970874, -1.7260171291897597)

In [21]:
keras.backend.set_learning_phase(0)
set_log_level(logging.DEBUG)

# CarliniWagner attack
from lib.custom_cw import CustomCarliniWagnerL2

attack_iterations = 200
cw_params = {'binary_search_steps': 3,
             'max_iterations': attack_iterations,
             'learning_rate': 0.01,
             'batch_size': n_attack,
             'initial_const': 1,
             'y_target': y_target}
wrap_clf = KerasModelWrapper(model)
cw = CustomCarliniWagnerL2(wrap_clf, featnet, thres=4*.75, sess=sess)
adv = cw.generate_np(X_atk, **cw_params)

[INFO 2018-10-30 06:48:54,506 cleverhans] Constructing new graph for attack CustomCarliniWagnerL2
[DEBUG 2018-10-30 06:48:56,364 cleverhans] Running CWL2 attack on instance 0 of 250
[DEBUG 2018-10-30 06:48:56,578 cleverhans]   Binary search step 0 of 3
[DEBUG 2018-10-30 06:48:58,562 cleverhans]     Iteration 0 of 200: loss=187 l2=0 loss1=2.75e-07
[DEBUG 2018-10-30 06:48:59,976 cleverhans]     Iteration 20 of 200: loss=52.5 l2=5.48 loss1=2.12
[DEBUG 2018-10-30 06:49:01,288 cleverhans]     Iteration 40 of 200: loss=24 l2=9.57 loss1=3.37
[DEBUG 2018-10-30 06:49:02,572 cleverhans]     Iteration 60 of 200: loss=16.1 l2=9.37 loss1=3.73
[DEBUG 2018-10-30 06:49:03,992 cleverhans]     Iteration 80 of 200: loss=12 l2=8.03 loss1=3.73
[DEBUG 2018-10-30 06:49:05,441 cleverhans]     Iteration 100 of 200: loss=9.39 l2=6.99 loss1=3.56
[DEBUG 2018-10-30 06:49:06,856 cleverhans]     Iteration 120 of 200: loss=7.86 l2=6.35 loss1=3.44
[DEBUG 2018-10-30 06:49:08,211 cleverhans]     Iteration 140 of 200: lo

In [22]:
featnet = FeatNet("featnet_sq3", [32, 32, 3], [1], 
                  [pos_S, pos_T, pos_O, pos_P], 
                  squeeze=3,
                  learning_rate=5e-4, 
                  save_path="model/featnet_sq3.h5",
                  load_model=True,
                  stn_weight=stn_weight)

In [23]:
y_adv = featnet.predict_model(sess, adv)
print(np.sum(y_adv >= 3))

153
